<a href="https://colab.research.google.com/github/cssc9cssc9/python_test/blob/main/FasterRCNNinOri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install ternausnet

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import re
import os

from PIL import Image

import albumentations as alb
from albumentations.pytorch.transforms import ToTensor

import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from glob import glob
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
import ternausnet.models
import warnings
warnings.warn('ignore')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: ignore


In [4]:
%cd /content/drive/MyDrive/訓練資料

/content/drive/MyDrive/訓練資料


In [5]:
%ls

'ground truth'/   id_list.pkl   original/


In [6]:
mask_dir = os.path.join(os.getcwd(),'ground truth')
img_dir = os.path.join(os.getcwd(), 'original')

In [7]:
import pickle
# id_list = [n.split('/')[-1] for n in glob(os.path.join(img_dir,'*.jpg'))]
# with open("id_list.pkl",'wb') as fp:
#   pickle.dump(id_list,fp)
with open("id_list.pkl",'rb') as fp:
  id_list = pickle.load(fp)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Explore Data Analysis

In [9]:
def url_mask(idx):
  return os.path.join(tar_dir, idx)
def url_img(idx):
  return os.path.join(img_dir, idx)

In [10]:
transform = alb.Compose(
    [
     ToTensor()
    ]
)
transform1 = alb.Compose(
    [
     alb.VerticalFlip(p=0.5),
     alb.RandomBrightness(0.3, p=1),
    #  alb.RandomCrop(256,256),
     ToTensor()
    ]
)
transform2 = alb.Compose(
    [
     alb.IAAAdditiveGaussianNoise(always_apply=True),
     ToTensor()
    ]
)
transform3 = alb.Compose(
    [
     alb.ShiftScaleRotate(shift_limit=0,rotate_limit=20,always_apply=True),
     ToTensor()
    ]
)
transform4 = alb.Compose(
    [
     alb.IAAAffine(shear=45),
     ToTensor()
    ]
)

#### Albumentations Transformation Test

In [11]:
# idx = id_list[0]
# img1 = cv2.cvtColor(cv2.imread(url_img(idx)),cv2.COLOR_BGR2RGB)
# img2 = cv2.cvtColor(cv2.imread(url_tar(idx)),cv2.COLOR_BGR2RGB)
# img2 = preprocessing_mask(img2)

# trans = transform1(image=img1, mask=img2)
# fig, ax = plt.subplots(2,2,figsize=(16,8))
# ax[0,0].imshow(trans['image'])
# ax[0,0].set_title('Augmented image')
# ax[0,0].set_axis_off()
# ax[0,1].imshow(trans['mask'])
# ax[0,1].set_axis_off()
# ax[0,1].set_title('Augmented mask')
# ax[1,0].imshow(img1)
# ax[1,1].imshow(img2)
# # ax.set_axis_off()
# plt.tight_layout()
# plt.show()

In [12]:
# next(zip(transform1(image=img1, mask=img2)))

#### Construct Model

In [11]:
params={
    "model": "UNet11",
    'lr':0.001,
    "batch_size":16,
    "num_workers":2,
    "epoch":2,
    "random_state":18,
    "device":device,
}

In [32]:
def preprocessing_mask(mask):
  make = mask.astype(np.float32)
  mask[mask<245] = 0
  mask[mask>=245] = 255
  return mask

In [22]:
class OriDataset(Dataset):
  def __init__(self,dir_list, transforms=None):
    super().__init__()
    self.filename = dir_list
    self.transforms = transforms

  def __getitem__(self, idx):
    image_file = self.filename[idx]

    image = cv2.imread(os.path.join('/content/drive/MyDrive/訓練資料/original', image_file))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

    mask = cv2.imread(os.path.join('/content/drive/MyDrive/訓練資料/ground truth', image_file),cv2.IMREAD_GRAYSCALE)
    mask = preprocessing_mask(mask)
    mask = mask
    if self.transforms:
      trans = self.transforms(image=image, mask=mask)
      image = trans['image']
      mask = trans['mask'].squeeze
      # assert(mask.shape==image.shape)

    return image, mask

  def __len__(self) -> int:
    return len(self.filename)

In [24]:
def create_dataloaders(id_list, params):
  tr_list, val_list = train_test_split(id_list, test_size=0.2, random_state = params['random_state'])
  tr_dataset0 = OriDataset(tr_list, transforms=transform)
  tr_dataset1 = OriDataset(tr_list, transforms=transform1)
  tr_dataset2 = OriDataset(tr_list, transforms=transform2)
  # tr_dataset3 = OriDataset(tr_list, transforms=transform3)
  # tr_dataset4 = OriDataset(tr_list, transforms=transform4)
  tr_dataset = torch.utils.data.ConcatDataset([tr_dataset0,tr_dataset1,tr_dataset2])

  val_dataset = OriDataset(val_list, transforms=transform)
  tr_dataloader = DataLoader(tr_dataset, batch_size=params['batch_size'], shuffle=False, num_workers=params['num_workers'], pin_memory=True)
  val_dataloader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False, num_workers=params['num_workers'], pin_memory=True)
  print(f'DataLoader is created! Training Dataloader size:{tr_dataset.__len__()}, Valid Dataloader size:{val_dataset.__len__()}\n')
  return tr_dataloader, val_dataloader

In [25]:
tr_dataloader, val_dataloader = create_dataloaders(id_list, params)

DataLoader is created! Training Dataloader size:65076, Valid Dataloader size:5424



#### Train Model

In [26]:
model = getattr(ternausnet.models, params["model"])(pretrained=True)
model = model.to(params["device"])

Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /root/.cache/torch/hub/checkpoints/vgg11-bbd30ac9.pth


In [27]:
class MetricMonitor:
  def __init__(self, float_precision=3):
    self.float_precision = float_precision
    self.reset()
  
  def reset(self):
    self.metrics = defaultdict(lambda: {"val":0, "count":0, "avg":0})
  
  def update(self, metric_name, val):
    metric = self.metrics[metric_name]

    metric["val"] += val
    metric["count"] += 1
    metric['avg'] = metric['val']/metric['count']

  def __str__(self):
    return " | ".join(
        [
         "{metric_name}: {avg:.{float_precision}f}".format(
             metric_name=metric_name, avg=metric['avg'], float_precision=self.float_precision
         )
         for (metric_name, metric) in self.metrics.items()
        ]
    )

In [28]:
def train(tr_dataloader, model, criterion, optimizer, epoch, params):
  metric_monitor = MetricMonitor()
  model.train()
  stream = tqdm(tr_dataloader)
  for i, (images, target) in enumerate(stream, start=1):
    images = images.to(params['device'], non_blocking=True)
    target = target.to(params['device'], non_blocking=True)
    output = model(images).squeeze()
    loss = criterion(output, target)
    metric_monitor.update("Loss", loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    stream.set_description(
        "Epoch: {epoch}. Train. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
    )

In [29]:
def validation(val_dataloader, model, criterion, epoch, params):
  metric_monitor = MetricMonitor()
  model.eval()
  stream = tqdm(val_dataloader)
  with torch.no_grad():
    for i, (image, target) in enumerate(stream, start=1):
      images = images.to(params['device'], non_blocking=True)
      target = target.to(params['device'], non_blocking=True)
      output = model(image).squeeze()
      loss = criterion(output, target)
      metric_monitor.update('Loss', loss.item())
      stream.set_description(
          "Epoch: {epoch}. Validation. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
      )

In [30]:
def tr_and_val(model, tr_dataloader, val_dataloader, params):
  criterion = torch.nn.BCEWithLogitsLoss().to(params['device'])
  optimizer = torch.optim.Adam(model.parameters(),lr=params['lr'])
  for epoch in range(1, params["epoch"]+1):
    train(tr_dataloader, model, criterion, optimizer, epoch, params)
    validation(val_dataloader, model, criterion,epoch, parms)
  return model

In [33]:
model = tr_and_val(model, tr_dataloader, val_dataloader, params)


  0%|          | 0/4068 [00:00<?, ?it/s]

TypeError: ignored